<a href="https://colab.research.google.com/github/Yuxuan-Zhang-Dexter/cuda-practice/blob/main/cuda_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Tue Jan  9 05:11:06 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [6]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [7]:
# Install the extension
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

# Load the extension into Colab
%load_ext nvcc_plugin

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-4v_pfisd
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-4v_pfisd
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0d2ab99cccbbc682722e708515fe9c4cfc50185a
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4716 sha256=455bb1a08aeb2f0c8d9081b68e6398a35aaeee9728ca66180923680b72b8d928
  Stored in directory: /tmp/pip-ephem-wheel-cache-2jmuhpii/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [11]:
### First Compile Cuda Code
%%cu
#include <iostream>
int main( void ) {
 printf( "Hello, World!\n" );
 return 0;
}

Hello, World!



In [13]:
### Kernal Call - kernel function runs on device, main function runs on host
%%cu
#include <iostream>

__global__ void kernel() {
    // Kernel code goes here
}

int main() {
    kernel<<<1,1>>>(); // Launch the kernel
    printf("Hello, World!\n");
    return 0;
}


Hello, World!



In [17]:
### passing parameter to the device
%%cu
#include <iostream>
__global__ void add( int a, int b, int *c ) {
 *c = a + b;
}
int main( void ) {
 int c;
 int *dev_c;
 cudaMalloc( (void**)&dev_c, sizeof(int) ); // allocate int space on the device
 add<<<1,1>>>( 2, 7, dev_c );
 cudaMemcpy( &c, dev_c, sizeof(int), cudaMemcpyDeviceToHost ); // read the content from device to host by using two pointers
 printf( "2 + 7 = %d\n", c );
 cudaFree( dev_c ); // cuda free device memory
 return 0;
}

2 + 7 = 9



In [21]:
### print device info
%%cu
#include <iostream>
int main (void) {
    int count;
    cudaGetDeviceCount(&count);
    for (int i = 0; i < count; i++) {
        cudaDeviceProp prop;
        cudaGetDeviceProperties(&prop, i);
        printf("Device Number: %d\n", i + 1);
        printf("  Device name: %s\n", prop.name);
        printf("  Memory Clock Rate (KHz): %d\n",
               prop.memoryClockRate);
        printf("  Memory Bus Width (bits): %d\n",
               prop.memoryBusWidth);
        printf("  Peak Memory Bandwidth (GB/s): %f\n\n",
               2.0*prop.memoryClockRate*(prop.memoryBusWidth/8)/1.0e6);
        // Add more properties you're interested in here.
    }
    return 0;
}

Device Number: 1
  Device name: Tesla T4
  Memory Clock Rate (KHz): 5001000
  Memory Bus Width (bits): 256
  Peak Memory Bandwidth (GB/s): 320.064000


